In [2]:

# coding: utf-8

#TODO put buttons in a class
#TODO add input box
#TODO other code impovments

# In[1]:


import pygame
from pygame.locals import *
import random
import math
import string


#Sources
#http://usingpython.com/pygame-intro/
#https://pythonprogramming.net/pygame-python-3-part-1-intro/


# In[2]:

#Constants Box
clock = pygame.time.Clock()

agent_radius =10
agent_speed=3
agent_color=(100,100,100)
agent_list =[]

width= 500
height =500
title="Title"

white = (255,255,255)
black = (0,0,0)
red = (255,0,0)
green =(0,255,0)
lime_green=(50,205,50)

button_width=100
button_height=60
button_x=20
button_y=20
button_space=10

ACCEPTED = string.digits

go_button= (button_x,button_y,button_width,button_height)
stop_button=(button_x+button_width+button_space,button_y,button_width,button_height)
TextBox_location=(button_x+2*(button_width+button_space),button_y,button_width,button_height)


#http://programarcadegames.com/python_examples/f.php?file=bouncing_balls.py
class Agent:

    #Initialize variables
    def __init__(self):
        self.x =0
        self.y =0
        self.change_x =0
        self.change_y =0
        self.color = agent_color

    def bounce(self):
        angle = random.uniform(0,2*math.pi)
        self.change_x =math.cos(angle)*agent_speed
        self.change_y =math.sin(angle)*agent_speed


def make_agent():
    agent=Agent()
    agent.x=random.randrange(agent_radius,width-agent_radius)
    agent.y=random.randrange(agent_radius+(button_y+button_height),height-agent_radius)

    agent.bounce()

    #make sure it's not 0
    while agent.change_x == agent.change_y ==0:
        print("oooops")
        pause()

    return agent


# In[3]:
#https://github.com/Mekire/pygame-textbox
#modified from above source
class TextBox(object):
    def __init__(self,rect):
        self.rect = pygame.Rect(rect)
        self.buffer = [] #this holds the input
        self.final = None #this is the current input displayed
        self.text_size =10

        self.rendered = None
        self.render_rect = None
        self.render_area = None

        self.blink = True
        self.blink_timer = 0.0

        defaults = {
                    "active" : False,
                    "color" : pygame.Color("white"),
                    "font_color" : pygame.Color("black"),
                    "outline_color" : pygame.Color("black"),
                    "outline_width" : 2,
                    "active_color" : pygame.Color("blue"),
                    "font" : pygame.font.Font("freesansbold.ttf",self.text_size),
                    "clear_on_enter" : True,
                    "inactive_on_enter" : True}

        self.__dict__.update(defaults)

    def get_event(self,event):
        if event.type == pygame.KEYDOWN and self.active:
            if event.key in (pygame.K_RETURN,pygame.K_KP_ENTER):
                self.execute()
            elif event.key == pygame.K_BACKSPACE:
                if self.buffer:
                    self.buffer.pop()
            elif event.unicode in ACCEPTED:
                self.buffer.append(event.unicode)
        elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            self.active = self.rect.collidepoint(event.pos)




    def execute(self):
        self.active = not self.inactive_on_enter
        if self.clear_on_enter:

            for i in range(int("".join(self.buffer))):
                agent_list.append(make_agent())

            self.buffer = []

    def update(self):
        new = "".join(self.buffer)
        if new != self.final:
            self.final = new
            self.rendered = self.font.render(self.final, True, self.font_color)
            self.render_rect = self.rendered.get_rect(x=self.rect.x+2,centery=self.rect.centery)
            if self.render_rect.width > self.rect.width-6:
                offset = self.render_rect.width-(self.rect.width-6)
                self.render_area = pygame.Rect(offset,0,self.rect.width-6,self.render_rect.height)
            else:
                self.render_area = self.rendered.get_rect(topleft=(0,0))
        if pygame.time.get_ticks()-self.blink_timer > 200:
            self.blink = not self.blink
            self.blink_timer = pygame.time.get_ticks()

    def draw(self,surface):
        outline_color = self.active_color if self.active else self.outline_color
        outline = self.rect.inflate(self.outline_width*2,self.outline_width*2)
        surface.fill(outline_color,outline)
        surface.fill(self.color,self.rect)
        if self.rendered:
            surface.blit(self.rendered,self.render_rect,self.render_area)
        if self.blink and self.active:
            curse = self.render_area.copy()
            curse.topleft = self.render_rect.topleft
            surface.fill(self.font_color,(curse.right+1,curse.y,2,curse.h))
# In[4]:



def text_objects(text, font):
    textSurface = font.render(text, True, black)
    return textSurface, textSurface.get_rect()

#http://cgp.wikidot.com/circle-to-circle-collision-detection
def collision(x1,y1,x2,y2):
    dx=x1-x2
    dy=y1-y2
    r=agent_radius+agent_radius
    if((dx*dx)+(dy*dy)<r*r):
        return True
    return False

def main():
    pygame.init()
    go_color=green
    run =True

    #Text Font
    smallText = pygame.font.Font("freesansbold.ttf",20)
    largeText = pygame.font.Font("freesansbold.ttf",50)

    text = TextBox(TextBox_location)

    def pause():
        paused = True
        TextSurf, TextRect = text_objects("Paused", largeText)
        TextRect.center = ((width/2),(height/2))
        Display.blit(TextSurf, TextRect)
        while paused:
            for event in pygame.event.get():

                if event.type == pygame.QUIT:
                    run=False
                    paused=False
                    #pygame.quit()
                    #quit()
                elif event.type == pygame.KEYDOWN:
                    #https://www.pygame.org/docs/ref/key.html
                    if event.key == pygame.K_g:
                        paused = False
            #TODO make buttons into functions to unpause
            pygame.display.update()

    def draw_go_button():

        #Button
        pygame.draw.rect(Display, go_color,go_button)

        #Go text (after button bc we draw it on top)
        textSurf, textRect = text_objects("GO!", smallText)
        textRect.center = ((button_x+(button_width/2)), (button_y+(button_height/2)) )
        Display.blit(textSurf, textRect)



    def draw_stop_button():
        #Button
        pygame.draw.rect(Display, red,(stop_button))

        #Stop text
        textSurf, textRect = text_objects("Stop!", smallText)
        textRect.center = ( (button_x+button_width+button_space+(button_width/2)), (button_y+(button_height/2)) )
        Display.blit(textSurf, textRect)


    Display = pygame.display.set_mode((width,height))
    pygame.display.set_caption(title)





    while run:
        for event in pygame.event.get():
            if event.type == QUIT:
                run=False
            elif event.type == pygame.KEYDOWN:
                #Make new agents by pressing space
                if event.key == pygame.K_SPACE:
                    agent_list.append(make_agent())

                #press p to pause game
                elif event.key == pygame.K_p:
                    pause()

                #check if input box is active
                if text.active:
                    if event.key in (pygame.K_RETURN,pygame.K_KP_ENTER):
                        text.execute()
                    #Delete last input
                    elif event.key == pygame.K_BACKSPACE:
                        if text.buffer:
                            text.buffer.pop()
                    #if input is accepted (only int are) then add it to end
                    elif event.unicode in ACCEPTED:
                        text.buffer.append(event.unicode)

            elif event.type == pygame.MOUSEBUTTONUP and event.button==1:
                #get mouse position
                mouse = pygame.mouse.get_pos()

                #check if the mouse is above the GO button
                if button_width+button_x > mouse[0] > button_x and button_height+button_y > mouse[1] > button_y:
                    go_color=lime_green
                    agent = make_agent()
                    agent_list.append(agent)

                #check if the mouse is above the STOP button
                elif button_x+2*button_width+button_space > mouse[0] > button_x+button_width+button_space and button_height+button_y > mouse[1] > button_y:
                    go_color=green
                    pause()

                #check if the mouse is above the input box
                else:
                    text.active = text.rect.collidepoint(event.pos)

        #list of dictionary of agents that have moved, reset each loop
        moved_agents =[]

        #Agent Logic
        for agent in agent_list:
            move=True
            agent.color=agent_color
            x=agent.x+agent.change_x
            y=agent.y+agent.change_y
            #check for collision with other agents
            for m in moved_agents:
                if(collision(x,y,m['x'],m['y'])):
                    m['agent'].color=lime_green
                    agent.color=lime_green

                    #calculate how to bounce the agents
                    distance = agent.x-m['x'], agent.y-m['y']
                    norm=math.sqrt(distance[0]**2+distance[1]**2)
                    direction=distance[0]/norm,distance[1]/norm


                    agent.change_x=direction[0]*agent_speed
                    agent.change_y=direction[1]*agent_speed

                    m['agent'].change_x=agent.change_x*-1
                    m['agent'].change_y=agent.change_y*-1

                    move=False
            moved_agents.append({'x':agent.x,'y':agent.y,'agent':agent})

            if(move):
                # Move the center
                agent.x += agent.change_x
                agent.y += agent.change_y

                #Stay in bounds
                if agent.y > height - agent_radius or agent.y < agent_radius+(button_y+button_height):
                    agent.change_y *= -1
                    agent.y+=agent.change_y

                if agent.x > width - (agent_radius+button_y) or agent.x < agent_radius:
                    agent.change_x *= -1
                    agent.x+=agent.change_x


        Display.fill(white)

        # Draw the agents
        for agent in agent_list:
            #the x and y cordinates are kept as floats, but to draw they need to be int
            pygame.draw.circle(Display, agent.color, [int(round(agent.x)), int(round(agent.y))], agent_radius)


        #draw the buttons
        draw_go_button()
        draw_stop_button()

        text.update()
        text.draw(Display)

        # Limit to 60 frames per second
        clock.tick(60)

        #update the screen
        pygame.display.flip()

    #end of loop we exit
    pygame.quit()
if __name__ == "__main__":
    main()
